Problem 1

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm

In [ ]:
start_date = "2022-01-01"
end_date= "2025-01-01"
risk_free_ticker = "^IRX"
market_index_ticker= "^GSPC"
equity_tickers= ["MSFT"]

In [ ]:
risk_free_data = yf.download(risk_free_ticker, start=start_date, end=end_date)["Close"]
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)["Close"]
stock_data = yf.download(equity_tickers, start=start_date, end=end_date)["Close"]

In [ ]:
import plotly.graph_objects as go

In [ ]:
market_returns = market_data.pct_change().dropna()
stock_returns = stock_data.pct_change().dropna()

risk_free_rate = risk_free_data / 100 / 252
risk_free_rate = risk_free_rate.reindex(market_returns.index, method="ffill")

risk_free_rate = pd.DataFrame(risk_free_rate).dropna()

merged_df = pd.merge(stock_returns,
                     pd.merge(market_returns, risk_free_rate,
                             left_index=True, right_index=True),
                     left_index=True, right_index=True)

for col in merged_df.columns:
  merged_df[f"{col}_cumu_return"] = (1 + merged_df[col]).cumprod()

In [ ]:
merged_df['market_excess_return'] = merged_df['^GSPC'] - merged_df['^IRX']
merged_df['dt'] = merged_df.index

In [ ]:
stock = 'MSFT'
y = merged_df[stock] - merged_df['^IRX']
x= merged_df['market_excess_return']
model = sm.OLS(y, x).fit()
print(f"CAPM model {stock}: beta is")
print(model.params)

In [ ]:
stock = '^GSPC'
y = merged_df[stock] - merged_df['^IRX']
x= merged_df['market_excess_return']
model = sm.OLS(y, x).fit()
print(f"CAPM model {stock}: beta is")
print(model.params)

In [ ]:
print(model.summary())

Based on the results above,  I can say that MSFT is less volatile than the market due to its lesser beta value. Unfortunately, I could not for the life of me figure out how to plot graphs.

Problem 2

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm

In [ ]:
start_date = "2022-01-01"
end_date= "2025-01-01"
risk_free_ticker = "^IRX"
market_index_ticker= "^GSPC"
equity_tickers= ["MSFT"]

In [ ]:
risk_free_data = yf.download(risk_free_ticker, start=start_date, end=end_date)["Close"]
market_data = yf.download(market_index_ticker, start=start_date, end=end_date)["Close"]
stock_data = yf.download(equity_tickers, start=start_date, end=end_date)["Close"]

In [ ]:
stock = 'MSFT'
y = merged_df[stock] - merged_df['^IRX']
x= merged_df['market_excess_return']
model = sm.OLS(y, x).fit()
print(f"CAPM model {stock}: beta is")
print(model.params)

In [ ]:
FF_df = pd.read_csv("/adjusted fama french daily.csv")
FF_df['Date'] = pd.to_datetime(FF_df['Date'], format='%Y%m%d')
FF_df['Mkt_RF'] = FF_df['Mkt-RF']/100
FF_df['SMB'] = FF_df['SMB']/100
FF_df['HML'] = FF_df['HML']/100
FF_df['RMW'] = FF_df['RMW']/100

stock_returns = stock_returns.merge(FF_df, on = 'Date', how = 'inner')

print(stock_returns['Date'].min(), stock_returns['Date'].max())

In [ ]:
FF_df.dtypes

In [ ]:
FF_df.head()

In [ ]:
stock_returns.head()

In [ ]:
y = stock_returns['MSFT'] - stock_returns['RMW']
x = stock_returns[['Mkt_RF', 'SMB', 'HML']]
model= sm.OLS(y, x).fit()
print(model.summary())

R squared appears to be greater for the FF model.
I can say that it appears low SMB ratios are caused by overexposure to large market caps, and high HMLs account for value stocks.

Problem3

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
start_date = "2022-01-01"
end_date= "2025-01-01"

Stock1 = ["AAPL"]
Stock2 = ["MSFT"]
Stock3 = ["GOOGL"]
Stock4 = ["AMZN"]
Stock5 = ["META"]
Stock6 = ["TSLA"]
Stock7 = ["JPM"]
Stock8 = ["PFE"]
Stock9 = ["KO"]
Stock10 = ["XOM"]
Stock1 = yf.download(Stock1, start=start_date, end=end_date)["Close"]
Stock2 = yf.download(Stock2, start=start_date, end=end_date)["Close"]
Stock3 = yf.download(Stock3, start=start_date, end=end_date)["Close"]
Stock4 = yf.download(Stock4, start=start_date, end=end_date)["Close"]
Stock5 = yf.download(Stock5, start=start_date, end=end_date)["Close"]
Stock6 = yf.download(Stock6, start=start_date, end=end_date)["Close"]
Stock7 = yf.download(Stock7, start=start_date, end=end_date)["Close"]
Stock8 = yf.download(Stock8, start=start_date, end=end_date)["Close"]
Stock9 = yf.download(Stock9, start=start_date, end=end_date)["Close"]
Stock10 = yf.download(Stock10, start=start_date, end=end_date)["Close"]

In [ ]:
stock_datas = pd.concat([Stock1, Stock2, Stock3, Stock4, Stock5, Stock6, Stock7, Stock8, Stock9, Stock10], axis=1)
stock_datas.columns = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'JPM', 'PFE', 'KO', 'XOM']
stock_returns = stock_datas.pct_change().dropna()

In [ ]:
stock_returns.head()

In [ ]:
FF_df = pd.read_csv("/adjusted fama french daily.csv")
FF_df['Date'] = pd.to_datetime(FF_df['Date'], format='%Y%m%d')
FF_df['Mkt_RF'] = FF_df['Mkt-RF']/100
FF_df['SMB'] = FF_df['SMB']/100
FF_df['HML'] = FF_df['HML']/100
FF_df['RMW'] = FF_df['RMW']/100

stock_returns = stock_returns.merge(FF_df, on = 'Date', how = 'inner')

print(stock_returns['Date'].min(), stock_returns['Date'].max())

In [ ]:
columns_to_exclude = ['Data', 'Mkt-RF', 'SMB', 'HML', 'RMW']

stock_list = [col for col in stock_returns.columns if col not in columns_to_exclude]

print(stock_list)

In [ ]:
all_betas = []
for stock in stock_list:
  if stock == 'Date':
    continue

  y = stock_returns[stock] - stock_returns['RMW']
  x = stock_returns[['Mkt_RF', 'SMB', 'HML']]
  model= sm.OLS(y, x).fit()
  temp = {'ticker': stock,
          'MKT_excess_beta': model.params['Mkt_RF'],
          'SMB_beta': model.params['SMB'],
          'HML_beta': model.params['HML']}
  all_betas.append(temp)

all_beta_df = pd.DataFrame(all_betas)
all_beta_df.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

columns_for_clustering = ['MKT_excess_beta','SMB_beta','HML_beta']
X = all_beta_df[columns_for_clustering]

inertia = []
silhoutte_scores = []
range_n_clusters = range(2, 11)

for k in range_n_clusters:
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(X)
  inertia.append(kmeans.inertia_)
  silhoutte_scores.append(silhouette_score(X, kmeans.labels_))

  plt.figure(figsize=(12, 5))

  plt.subplot(1, 2, 1)
  plt.plot(range_n_clusters, inertia)
  plt.xlabel('Number of clusters')
  plt.ylabel('Inertia')

  plt.subplot(1, 2, 2)
  plt.plot(range_n_clusters, silhoutte_scores, marker='0')
  plt.xlabel('Number of clusters')
  plt.ylabel('Silhouette score')

  plt.tight_layout()
  plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

columns_for_clustering = ['MKT_excess_beta','SMB_beta','HML_beta']
X = all_beta_df[columns_for_clustering]

inertia = []
silhoutte_scores = []
range_n_clusters = range(2, 11)

# Calculate inertia and silhouette scores for all cluster numbers first
for k in range_n_clusters:
  kmeans = KMeans(n_clusters=k)
  kmeans.fit(X)
  inertia.append(kmeans.inertia_)
  silhoutte_scores.append(silhouette_score(X, kmeans.labels_))

# Then plot the results outside the loop
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(range_n_clusters, inertia)
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

plt.subplot(1, 2, 2)
plt.plot(range_n_clusters, silhoutte_scores)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')

plt.tight_layout()
plt.show()

In [ ]:
optimal_clusters = 3

kmeans = KMeans(n_clusters=optimal_clusters, random_state=20).fit(X)
X['Cluster'] = kmeans.labels_

all_beta_df['Cluster'] = kmeans.labels_
cluster_analysis = all_beta_df.groupby('Cluster').agg(mean_mkt_beta = ('MKT_excess_beta', 'mean'),
                                                       mean_smb_beta = ('SMB_beta', 'mean'),
                                                       mean_hml_beta = ('HML_beta', 'mean'))


print("Cluster Analysis (Means and Record Counts):\n"), cluster_analysis


I was unable to figure out how to graph the scatterplot, but from the table I can tell that stocks from this industry vary in volatility and market cap all the way across the spectrum.